In [1]:
import requests
from bs4 import BeautifulSoup
import re
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
def find_contact_page(site_url):
  response = requests.get(site_url)
  soup = BeautifulSoup(response.content, "html.parser")
  links = soup.find_all("a", href=True)
  output = [link for link in links  if \
            'contact us' in str(link).lower() or  \
            'contact.' in str(link).lower() \
            ]
  return re.findall('"(.*?)"',str(output[0]))[0]

In [3]:
def scrape_contacts(contact_url):
  print(contact_url)
  response = requests.get(contact_url)
  soup = BeautifulSoup(response.content, "html.parser")
  return soup.text

In [4]:
base_url = "https://www.mouthshut.com/"

contact_page = find_contact_page(base_url)
contact_page
# if contact_page:
#   contact_details = [scrape_contacts(str(i).split('"')[1]) for i in contact_page]
#   print(contact_details)
# else:
#   print("Not found in {}".format(base_url))

'https://www.mouthshut.com/help/contact.php'

In [5]:
# llm performing extraction

In [6]:
model_name = "Qwen/Qwen2.5-0.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [18]:
page_source = BeautifulSoup(requests.get(contact_page).content, "html.parser")
prompt = """You are an expert webscraper. Fromt he following website page source, extract all possible contact of this website, like email, phone number and address
<URL>
{}
</URL>
""".format(str(page_source)[5000:10000])
messages = [{"role": "Get the link for contacts page of a website", "content": ""},{"role": "user", "content": prompt}]

In [19]:
%%time
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


CPU times: user 19.8 s, sys: 139 ms, total: 19.9 s
Wall time: 22.7 s
